In [1]:
import pytorch_lightning as pl
from typing import Optional, Dict, List, Union
from torchmetrics import Metric
import copy
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf, open_dict
from torch import nn
import torch
import hydra
from e3nn import o3
from curator.data import properties, Asap3NeighborList, AseDataReader
from curator.model import LitNNP, NeuralNetworkPotential
from curator.simulator import calculator
from curator.layer import scatter_add
from collections import OrderedDict
from pathlib import Path
import numpy as np
from torchmetrics import Metric

from ase.io import read, write
import curator
import mace
from typing import Callable
from dataclasses import dataclass
from typing import Final, Dict, Set

In [2]:
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from e3nn.util.jit import compile, compile_mode, script
from e3nn.util.test import assert_auto_jitable
from PaiNN.model import PainnModel
from PaiNN.data import AseDataset, collate_atomsdata
import torchmetrics
import logging

from nequip.data import dataset_from_config, DataLoader, AtomicData, AtomicDataDict
from torch_runstats.scatter import scatter
import yaml
from curator.utils import read_user_config
from collections import defaultdict
from ase.data import chemical_symbols

In [3]:
from nequip.scripts.train import parse_command_line
from nequip.train.trainer import Trainer
from nequip.data import dataset_from_config
from nequip.model import model_from_config
import pickle
from time import perf_counter
from nequip.utils import atomic_write_group
from nequip.data import AtomicData

In [4]:
config = parse_command_line(['nequip_example.yaml'])

In [5]:
trainer = Trainer(model=None, **dict(config))
# dataset = dataset_from_config(config, prefix="dataset")
dataset = pickle.load(open('/home/xinyang/nequip_aspirin.pl', mode='rb'))
trainer.set_dataset(dataset, None)
final_model = model_from_config(
    config=config, initialize=True, dataset=trainer.dataset_train
)
trainer.model = final_model

Torch device: cuda


/home/xinyang/miniconda3/lib/python3.11/site-packages/torch/jit/_check.py:172: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
Atomic outputs are scaled by: [H, C, O: None], shifted by [H, C, O: None].
!!!! Careful global_shift is set to dataset_total_energy_mean.The model for ['total_energy'] will no longer be size extensive
Replace string dataset_forces_rms to 25.22938346862793
Replace string dataset_total_energy_mean to tensor([-406740.3125])
Initially outputs are globally scaled by: 25.22938346862793, total_energy are globally shifted by tensor([-406740.3125]).


In [6]:
config.update(trainer.params)
trainer.update_kwargs(config)

In [7]:
# train initialization
trainer.init()
for callback in trainer._init_callbacks:
    callback(trainer)

trainer.init_log()
trainer.wall = perf_counter()
trainer.previous_cumulative_wall = trainer.cumulative_wall

with atomic_write_group():
    if trainer.iepoch == -1:
        trainer.save()
    if trainer.iepoch in [-1, 0]:
        trainer.save_config()
        
trainer.init_metrics()

Number of weights: 392824
Number of trainable weights: 392824
! Starting training ...


In [14]:
trainer.epoch_step()


training
# Epoch batch         loss       loss_f       loss_e        f_mae       f_rmse      H_f_mae      C_f_mae      O_f_mae  psavg_f_mae     H_f_rmse     C_f_rmse     O_f_rmse psavg_f_rmse        e_mae      e/N_mae
      1     1        0.472        0.352         0.12         9.97           15         5.15         10.3         18.9         11.4         7.23         14.5         24.5         15.4         8.73        0.416
      1     2        0.505        0.474       0.0306         12.1         17.4         8.91         11.3         20.5         13.5         13.6         16.4         24.7         18.2         4.31        0.205
      1     3        0.821        0.657        0.164         14.4         20.5         12.3         12.7         22.5         15.8         17.3         19.4         27.4         21.4         10.2        0.485
      1     4        0.844        0.718        0.126         15.6         21.4         12.1         15.3         23.5         16.9         16.2         21

In [4]:
from torch_ema import ExponentialMovingAverage
from curator.utils import EarlyStopping
import contextlib

In [5]:
cfg = read_user_config("null.yaml", config_path="../curator/configs")

In [5]:
model = instantiate(cfg.model)

/home/xinyang/miniconda3/lib/python3.11/site-packages/torch/jit/_check.py:172: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


In [6]:
task = instantiate(cfg.task, model=model)

In [13]:
atoms = read('/home/xinyang/Au-water_test.traj')

In [14]:
from curator.data import TorchNeighborList, AseDataReader
import curator

In [15]:
ase_reader = AseDataReader(5.0, transforms=[TorchNeighborList(5.0, return_distance=True)])

In [17]:
atoms_dict = ase_reader(atoms)
edge_dist = atoms_dict['_edge_distance']

In [21]:
from curator.layer import RadialBasisEdgeEncoding
from curator.layer import CosineCutoff, SineBasis

In [22]:
cutoff_fn = CosineCutoff(5.0)

In [23]:
sine = SineBasis(cutoff=5.0, num_basis=6)

In [25]:
filter = nn.Linear(6, 128 * 3)

In [17]:
sine(edge_dist)

tensor([[ 0.1311, -0.2212,  0.2420, -0.1870,  0.0734,  0.0631],
        [ 0.0204, -0.0407,  0.0605, -0.0798,  0.0982, -0.1157],
        [ 0.0106, -0.0212,  0.0317, -0.0421,  0.0524, -0.0626],
        ...,
        [ 0.0685, -0.1304,  0.1795, -0.2112,  0.2222, -0.2115],
        [ 0.0685, -0.1304,  0.1795, -0.2111,  0.2222, -0.2115],
        [ 0.1324, -0.2227,  0.2419, -0.1841,  0.0676,  0.0705]])

In [26]:
edge_dist = atoms_dict['_edge_distance']
filter_weight = filter(sine(edge_dist))
filter_weight = filter_weight * cutoff_fn(edge_dist).unsqueeze(-1)

In [27]:
sine(edge_dist) * cutoff_fn(edge_dist)[:, None]

tensor([[ 1.0266e-02, -1.7318e-02,  1.8945e-02, -1.4640e-02,  5.7498e-03,
          4.9411e-03],
        [ 5.0167e-05, -9.9841e-05,  1.4854e-04, -1.9577e-04,  2.4108e-04,
         -2.8403e-04],
        [ 7.1897e-06, -1.4360e-05,  2.1491e-05, -2.8564e-05,  3.5559e-05,
         -4.2458e-05],
        ...,
        [ 1.6719e-03, -3.1806e-03,  4.3790e-03, -5.1503e-03,  5.4190e-03,
         -5.1591e-03],
        [ 1.6706e-03, -3.1784e-03,  4.3762e-03, -5.1472e-03,  5.4163e-03,
         -5.1573e-03],
        [ 1.0553e-02, -1.7743e-02,  1.9278e-02, -1.4668e-02,  5.3843e-03,
          5.6158e-03]])

In [40]:
sine(edge_dist)[0,:] * cutoff_fn(edge_dist)[0]

tensor([ 0.0103, -0.0173,  0.0189, -0.0146,  0.0057,  0.0049])

In [28]:
test_input = sine(edge_dist)

In [38]:
cutoff_fn(edge_dist)

tensor([0.0783, 0.0025, 0.0007,  ..., 0.0244, 0.0244, 0.0797])

In [36]:
filter(sine(edge_dist)) * cutoff_fn(edge_dist).unsqueeze(-1)

tensor([[-2.1693e-03, -3.3071e-02, -1.3626e-02,  ...,  3.0264e-02,
          3.9087e-03,  2.3733e-02],
        [-1.2179e-04, -9.6975e-04, -6.2630e-04,  ...,  9.2199e-04,
          5.5822e-04,  7.2841e-04],
        [-3.3015e-06, -2.6811e-04, -1.5680e-04,  ...,  2.5786e-04,
          1.6555e-04,  1.7821e-04],
        ...,
        [-4.0805e-03, -9.8653e-03, -7.3949e-03,  ...,  9.0350e-03,
          3.4794e-03,  9.4409e-03],
        [-4.0785e-03, -9.8605e-03, -7.3916e-03,  ...,  9.0307e-03,
          3.4785e-03,  9.4364e-03],
        [-1.7919e-03, -3.3655e-02, -1.3551e-02,  ...,  3.0806e-02,
          3.8812e-03,  2.3921e-02]], grad_fn=<MulBackward0>)

In [8]:
irreps_in = o3.Irreps([(6, (0, 1))])
irreps_out = o3.Irreps([(128, (0, 1))])

In [ ]:
cutoff_fn(edge_dist)

In [10]:
linear = o3.Linear(irreps_in, irreps_out)

/home/xinyang/miniconda3/lib/python3.11/site-packages/torch/jit/_check.py:172: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


In [31]:
linear(test_input).shape

torch.Size([9838, 128])

In [32]:
linear.state_dict()

OrderedDict([('weight',
              tensor([ 9.9679e-01,  2.6505e-02,  1.9578e-01, -9.3789e-01,  5.7741e-01,
                      -1.4304e-01,  2.0971e-01, -1.1517e-01, -4.1923e-01, -1.8534e+00,
                       1.2301e+00, -6.4429e-01,  1.3929e-02, -1.2637e+00, -8.7034e-02,
                       1.4234e+00,  4.5670e-01, -7.9674e-02,  3.9620e-01, -5.5135e-01,
                       2.7207e-01, -1.8287e+00,  1.1931e+00,  5.1764e-01, -7.6357e-01,
                       1.4704e-01, -1.5680e-01, -9.4827e-02,  5.3387e-01,  5.0387e-01,
                       9.6885e-01,  1.1902e+00,  1.0256e+00, -7.7731e-01, -6.8786e-01,
                       1.1444e+00,  6.3259e-01,  8.4692e-02, -1.3049e-01, -7.2595e-01,
                       1.1002e+00, -6.0119e-01, -1.4631e-01, -8.1769e-01, -2.0363e+00,
                      -3.6261e-01,  3.5445e-02, -6.1868e-01,  5.4913e-01, -2.8015e-01,
                      -1.9248e+00,  9.6674e-01,  1.2315e-01,  1.0276e+00, -2.3247e-01,
                   

In [11]:
linear

Linear(6x0e -> 128x0e | 768 weights)

In [41]:
filter(sine(edge_dist)) * cutoff_fn(edge_dist).unsqueeze(-1)

tensor([[-2.1693e-03, -3.3071e-02, -1.3626e-02,  ...,  3.0264e-02,
          3.9087e-03,  2.3733e-02],
        [-1.2179e-04, -9.6975e-04, -6.2630e-04,  ...,  9.2199e-04,
          5.5822e-04,  7.2841e-04],
        [-3.3015e-06, -2.6811e-04, -1.5680e-04,  ...,  2.5786e-04,
          1.6555e-04,  1.7821e-04],
        ...,
        [-4.0805e-03, -9.8653e-03, -7.3949e-03,  ...,  9.0350e-03,
          3.4794e-03,  9.4409e-03],
        [-4.0785e-03, -9.8605e-03, -7.3916e-03,  ...,  9.0307e-03,
          3.4785e-03,  9.4364e-03],
        [-1.7919e-03, -3.3655e-02, -1.3551e-02,  ...,  3.0806e-02,
          3.8812e-03,  2.3921e-02]], grad_fn=<MulBackward0>)

In [32]:
cutoff_fn(edge_dist)

tensor([0.0783, 0.0025, 0.0007,  ..., 0.0244, 0.0244, 0.0797])

In [ ]:
filter

In [31]:
filter_weight

tensor([[-2.1693e-03, -3.3071e-02, -1.3626e-02,  ...,  3.0264e-02,
          3.9087e-03,  2.3733e-02],
        [-1.2179e-04, -9.6975e-04, -6.2630e-04,  ...,  9.2199e-04,
          5.5822e-04,  7.2841e-04],
        [-3.3015e-06, -2.6811e-04, -1.5680e-04,  ...,  2.5786e-04,
          1.6555e-04,  1.7821e-04],
        ...,
        [-4.0805e-03, -9.8653e-03, -7.3949e-03,  ...,  9.0350e-03,
          3.4794e-03,  9.4409e-03],
        [-4.0785e-03, -9.8605e-03, -7.3916e-03,  ...,  9.0307e-03,
          3.4785e-03,  9.4364e-03],
        [-1.7919e-03, -3.3655e-02, -1.3551e-02,  ...,  3.0806e-02,
          3.8812e-03,  2.3921e-02]], grad_fn=<MulBackward0>)

In [30]:
filter(edge_dist_embedding)

tensor([[ 0.0407, -0.3974, -0.2006,  ...,  0.3852,  0.2449,  0.2296],
        [ 0.0462, -0.3953, -0.2030,  ...,  0.3851,  0.2614,  0.2235],
        [ 0.0464, -0.3953, -0.2029,  ...,  0.3851,  0.2614,  0.2233],
        ...,
        [ 0.0413, -0.3955, -0.2053,  ...,  0.3847,  0.2585,  0.2273],
        [ 0.0413, -0.3955, -0.2053,  ...,  0.3847,  0.2586,  0.2273],
        [ 0.0410, -0.3975, -0.2002,  ...,  0.3852,  0.2445,  0.2294]],
       grad_fn=<AddmmBackward0>)

In [20]:
dist_embedding = RadialBasisEdgeEncoding(basis=sine, cutoff_fn=cutoff_fn)

In [24]:
edge_dist_embedding = dist_embedding(atoms_dict)['_edge_dist_embedding']

In [25]:
edge_dist_embedding

tensor([[ 1.0266e-02, -1.7318e-02,  1.8945e-02, -1.4640e-02,  5.7498e-03,
          4.9411e-03],
        [ 5.0167e-05, -9.9841e-05,  1.4854e-04, -1.9577e-04,  2.4108e-04,
         -2.8403e-04],
        [ 7.1897e-06, -1.4360e-05,  2.1491e-05, -2.8564e-05,  3.5559e-05,
         -4.2458e-05],
        ...,
        [ 1.6719e-03, -3.1806e-03,  4.3790e-03, -5.1503e-03,  5.4190e-03,
         -5.1591e-03],
        [ 1.6706e-03, -3.1784e-03,  4.3762e-03, -5.1472e-03,  5.4163e-03,
         -5.1573e-03],
        [ 1.0553e-02, -1.7743e-02,  1.9278e-02, -1.4668e-02,  5.3843e-03,
          5.6158e-03]])

In [27]:
filter(edge_dist_embedding).shape

torch.Size([9838, 384])

In [41]:
import abc

In [42]:
class RadialBasis(torch.nn.Module, metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def forward(self):
        pass

In [44]:
class SineBasis(RadialBasis):
    """
    calculate sinc radial basis function:
    
    sin(n *pi*d/d_cut)/d
    """
    def __init__(self, num_basis: int, cutoff: float):
        super().__init__()
        self.num_basis = num_basis
        self.cutoff = cutoff
        
    def forward(self, edge_dist: torch.Tensor) -> torch.Tensor:
        n = torch.arange(self.num_basis, device=edge_dist.device) + 1
        radial_basis = torch.sin(edge_dist.unsqueeze(-1) * n * torch.pi / self.cutoff) / edge_dist.unsqueeze(-1)
        return radial_basis

In [46]:
sine = SineBasis(6, 5.0)

In [20]:
atoms_dict = edge_dist_embedding(atoms_dict)

In [24]:
atoms_dict['_edge_dist_embedding']

tensor([[ 8.4926e-03, -1.4326e-02,  1.5672e-02,  ...,  4.0875e-03,
         -1.1651e-02,  1.5566e-02],
        [ 1.2768e-05, -2.5411e-05,  3.7804e-05,  ..., -7.2290e-05,
          8.2511e-05, -9.1921e-05],
        [ 1.0118e-06, -2.0208e-06,  3.0243e-06,  ..., -5.9748e-06,
          6.9294e-06, -7.8652e-06],
        ...,
        [ 1.0398e-03, -1.9782e-03,  2.7236e-03,  ..., -3.2088e-03,
          2.7341e-03, -1.9926e-03],
        [ 1.0389e-03, -1.9765e-03,  2.7214e-03,  ..., -3.2072e-03,
          2.7333e-03, -1.9929e-03],
        [ 8.7540e-03, -1.4718e-02,  1.5991e-02,  ...,  4.6585e-03,
         -1.2299e-02,  1.6019e-02]], grad_fn=<MulBackward0>)

In [22]:
a = (1, 2, 3)

In [23]:
a.

AttributeError: 'tuple' object has no attribute 'append'

In [18]:
edge_dist_embedding(atoms_dict)

KeyError: '_edge_dist'

In [47]:
optim = instantiate(cfg.task.optimizer)

In [48]:
optim.keywords['weight_decay']

0.01

In [18]:
from curator.model import MACE

In [19]:
type(model.representation) == MACE

True

In [16]:
type(model.representation)

curator.model.mace.MACE

In [58]:
cfg.model.representation

{'_target_': 'curator.model.PainnModel', 'num_interactions': 3, 'num_features': 128, 'cutoff': '${data.cutoff}', 'num_basis': 20, 'cutoff_fn': {'_target_': 'curator.layer.CosineCutoff', 'cutoff': '${data.cutoff}'}, 'radial_basis': {'_target_': 'curator.layer.SineBasis', 'cutoff': '${data.cutoff}', 'num_basis': '${model.representation.num_basis}'}}

In [59]:
cfg.task.optimizer

{'_target_': 'torch.optim.AdamW', '_partial_': True, 'lr': 0.0005, 'eps': 1e-08, 'betas': [0.9, 0.999], 'weight_decay': 0.01}

In [5]:
# Set up datamodule and load training and validation set
cfg = read_user_config("nequip.yaml", config_path="../curator/configs")
print(cfg.data.datapath)
if not os.path.isfile(cfg.data.datapath):
    raise RuntimeError("Please provide valid data path!")
datamodule = instantiate(cfg.data)
datamodule.setup()
train_loader = datamodule.train_dataloader
val_loader = datamodule.val_dataloader

# Set up the model, the optimizer and  the scheduler
model = instantiate(cfg.model)
model.initialize_modules(datamodule)
outputs = instantiate(cfg.task.outputs)
model.to(cfg.device)
for output in outputs:
    output.to(cfg.device)
optimizer = instantiate(cfg.task.optimizer)(model.parameters())
scheduler = instantiate(cfg.task.scheduler)(optimizer=optimizer)

# If you use exponentialmovingaverage, you need to update the model parameters
ema = ExponentialMovingAverage(model.parameters(), decay=0.995)

# Setup early stopping and inital training param  
early_stop = EarlyStopping(patience=cfg.patience) 
epoch = cfg.trainer.max_epochs
best_val_loss = torch.inf
prev_loss = None
rescale_layers = []
for layer in model.output_modules:
    if hasattr(layer, "unscale"):
        rescale_layers.append(layer)

e = 0

ConfigCompositionException: Error resolving interpolation '${model.representation}_optim', possible interpolation keys: data, model, model/representation, task, task/scheduler, task/optimizer

In [11]:
model.representation.embeddings.chemical_embedding.load_state_dict(trainer.model.model.func.chemical_embedding.state_dict())
model.representation.interactions[0].load_state_dict(trainer.model.model.func.layer0_convnet.state_dict(), strict=False)
model.representation.interactions[1].load_state_dict(trainer.model.model.func.layer1_convnet.state_dict(), strict=False)
model.representation.interactions[2].load_state_dict(trainer.model.model.func.layer2_convnet.state_dict(), strict=False)
model.representation.interactions[3].load_state_dict(trainer.model.model.func.layer3_convnet.state_dict(), strict=False)

model.representation.interactions[1].conv.avg_num_neighbors[0] = trainer.model.model.func.layer1_convnet.conv.avg_num_neighbors
model.representation.interactions[2].conv.avg_num_neighbors[0] = trainer.model.model.func.layer2_convnet.conv.avg_num_neighbors
model.representation.interactions[3].conv.avg_num_neighbors[0] = trainer.model.model.func.layer3_convnet.conv.avg_num_neighbors

model.representation.readout_mlp[0].load_state_dict(trainer.model.model.func.conv_to_output_hidden.linear.state_dict())
model.representation.readout_mlp[1].load_state_dict(trainer.model.model.func.output_hidden_to_scalar.linear.state_dict())

model.output_modules[2].scale_by[0] = trainer.model.scale_by
model.output_modules[2].shift_by = trainer.model.shift_by

In [13]:
# train
if e > 0:
    model.train()
    for i, batch in enumerate(train_loader):
        # Initialize the batch, targets and loss
        batch = {k: v.to(config.device) for k, v in batch.items()}
        targets = {
            output.target_property: batch[output.target_property]
            for output in outputs
        }
        atoms_dict = {k: v for k, v in batch.items() if k not in targets}
        optimizer.zero_grad()
        unscaled_targets = targets.copy()
        unscaled_targets.update(atoms_dict)
        for layer in rescale_layers:
            unscaled_targets = layer.unscale(unscaled_targets, force_process=True)
        pred = model(batch)
        loss = 0.0
        
        # Calculate the loss and metrics
        metrics = {}
        for output in outputs:
            tmp_loss, _ = output.calculate_loss(unscaled_targets, pred)
            metrics[f"{output.target_property}_loss"] = tmp_loss.detach()
            loss += tmp_loss
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        ema.update()
        
        # Log the training metrics
        scaled_pred = pred.copy()
        scaled_pred.update(atoms_dict)
        for layer in rescale_layers:
            scaled_pred = layer.scale(scaled_pred, force_process=True)
        if i % cfg.trainer.log_every_n_steps == 0:
            for output in outputs:
                for k, v in output.metrics['train'].items():
                    metrics[f"{output.name}_{k}"] = v(scaled_pred[output.name], targets[output.name]).detach()
            log_outputs = ",".join([f"{k}: {v:8.3f} " for k, v in metrics.items()])
            logging.info(f"Training step: {i} {log_outputs}")

# validation for each epoch
model.eval()
metrics = {}
a_counts = 0
s_counts = 0
cm = ema.average_parameters()
with cm:
    for i, batch in enumerate(val_loader):
        # Initialize the batch, targets and loss
        batch = {k: v.to(config.device) for k, v in batch.items()}
        targets = {
            output.target_property: batch[output.target_property]
            for output in outputs
        }
        atoms_dict = {k: v for k, v in batch.items() if k not in targets}
            
        a = batch["forces"].shape[0]
        s = batch["energy"].shape[0]
        a_counts += a
        s_counts += s
        pred = model(batch)
        
        unscaled_targets, unscaled_pred = targets.copy(), pred.copy()
        unscaled_pred.update(atoms_dict)
        unscaled_targets.update(atoms_dict)
        for layer in rescale_layers:
            unscaled_targets = layer.unscale(unscaled_targets, force_process=True)
            unscaled_pred = layer.unscale(unscaled_pred, force_process=True)
            
        # calculate loss
        for output in outputs:
            tmp_loss = output.calculate_loss(unscaled_targets, unscaled_pred, return_num_obs=False).detach()
            # metrics
            if i == 0:
                metrics[f"{output.target_property}_loss"] = tmp_loss
            else:
                metrics[f"{output.target_property}_loss"] += tmp_loss
                
            for k, v in output.metrics['train'].items():
                m = v(pred[output.name], targets[output.name]).detach()
                if "rmse" in k:
                    m = m ** 2
                if i == 0:
                    metrics[f"{output.name}_{k}"] = m
                else:
                    metrics[f"{output.name}_{k}"] += m
    
# postprocess validation metrics    
for k in metrics:
    metrics[k] /= i+1
    if "rmse" in k:
        metrics[k] = torch.sqrt(metrics[k])

val_loss = metrics["energy_loss"] + metrics["forces_loss"]
smooth_loss = val_loss if prev_loss is None else 0.9 * val_loss + 0.1 * prev_loss
prev_loss = val_loss

log_outputs = ",".join([f"{k}: {v:<8.3f} " for k, v in metrics.items()])
logging.info(f"Validation epoch: {e}, {log_outputs}, patience: {early_stop.counter}")
e += 1

Training step: 0 energy_loss:    0.120 ,forces_loss:    0.352 ,energy_mae:    8.734 ,energy_rmse:    8.738 ,forces_mae:    9.975 ,forces_rmse:   14.978 
Training step: 10 energy_loss:    0.013 ,forces_loss:    0.970 ,energy_mae:    2.734 ,energy_rmse:    2.876 ,forces_mae:   18.932 ,forces_rmse:   24.844 
Training step: 20 energy_loss:    0.041 ,forces_loss:    1.016 ,energy_mae:    5.109 ,energy_rmse:    5.122 ,forces_mae:   19.690 ,forces_rmse:   25.431 
Training step: 30 energy_loss:    0.001 ,forces_loss:    0.962 ,energy_mae:    0.594 ,energy_rmse:    0.601 ,forces_mae:   18.699 ,forces_rmse:   24.746 
Training step: 40 energy_loss:    0.115 ,forces_loss:    1.280 ,energy_mae:    8.531 ,energy_rmse:    8.535 ,forces_mae:   21.401 ,forces_rmse:   28.539 
Validation epoch: 1, energy_loss: 0.026    ,energy_mae: 3.421    ,energy_rmse: 4.053    ,forces_loss: 1.011    ,forces_mae: 18.267   ,forces_rmse: 25.369   , patience: 0


In [14]:
cr_test = next(iter(datamodule.train_dataloader))
np_test = AtomicData.to_AtomicDataDict(next(iter(trainer.dl_train)))

cr_test = {k: v.to("cuda") for k, v in cr_test.items()}
np_test = {k: v.to("cuda") for k, v in np_test.items()}

In [15]:
model.to("cuda")
trainer.model.to("cuda")

RescaleOutput(
  (model): GradientOutput(
    (func): SequentialGraphNetwork(
      (one_hot): OneHotAtomEncoding()
      (spharm_edges): SphericalHarmonicEdgeAttrs(
        (sh): SphericalHarmonics()
      )
      (radial_basis): RadialBasisEdgeEncoding(
        (basis): BesselBasis()
        (cutoff): PolynomialCutoff()
      )
      (chemical_embedding): AtomwiseLinear(
        (linear): Linear(3x0e -> 32x0e | 96 weights)
      )
      (layer0_convnet): ConvNetLayer(
        (equivariant_nonlin): Gate (96x0e+32x1o+32x2e -> 32x0e+32x2e+32x1o)
        (conv): InteractionBlock(
          (linear_1): Linear(32x0e -> 32x0e | 1024 weights)
          (fc): FullyConnectedNet[8, 64, 64, 96]
          (tp): TensorProduct(32x0e x 1x0e+1x1o+1x2e -> 32x0e+32x1o+32x2e | 96 paths | 96 weights)
          (linear_2): Linear(32x0e+32x1o+32x2e -> 96x0e+32x1o+32x2e | 5120 weights)
          (sc): FullyConnectedTensorProduct(32x0e x 3x0e -> 96x0e+32x1o+32x2e | 9216 paths | 9216 weights)
        )
      

In [16]:
np_rescale = []
outer_layer = trainer.model
while hasattr(outer_layer, "unscale"):
    np_rescale.append(outer_layer)
    outer_layer = getattr(outer_layer, "model", None)
    
cr_rescale = [model.output_modules[2]]

In [17]:
unscaled_np = np_test.copy()
unscaled_cr = cr_test.copy()
trainer.model.train()

RescaleOutput(
  (model): GradientOutput(
    (func): SequentialGraphNetwork(
      (one_hot): OneHotAtomEncoding()
      (spharm_edges): SphericalHarmonicEdgeAttrs(
        (sh): SphericalHarmonics()
      )
      (radial_basis): RadialBasisEdgeEncoding(
        (basis): BesselBasis()
        (cutoff): PolynomialCutoff()
      )
      (chemical_embedding): AtomwiseLinear(
        (linear): Linear(3x0e -> 32x0e | 96 weights)
      )
      (layer0_convnet): ConvNetLayer(
        (equivariant_nonlin): Gate (96x0e+32x1o+32x2e -> 32x0e+32x2e+32x1o)
        (conv): InteractionBlock(
          (linear_1): Linear(32x0e -> 32x0e | 1024 weights)
          (fc): FullyConnectedNet[8, 64, 64, 96]
          (tp): TensorProduct(32x0e x 1x0e+1x1o+1x2e -> 32x0e+32x1o+32x2e | 96 paths | 96 weights)
          (linear_2): Linear(32x0e+32x1o+32x2e -> 96x0e+32x1o+32x2e | 5120 weights)
          (sc): FullyConnectedTensorProduct(32x0e x 3x0e -> 96x0e+32x1o+32x2e | 9216 paths | 9216 weights)
        )
      

In [26]:
trainer.optim

Adam (
Parameter Group 0
    amsgrad: True
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.005
    maximize: False
    weight_decay: 0
)

In [27]:
optimizer

AdamW (
Parameter Group 0
    amsgrad: True
    betas: [0.9, 0.999]
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0.01
)

In [18]:
for layer in np_rescale:
    unscaled_np = layer.unscale(unscaled_np, force_process=True)

for layer in cr_rescale:
    unscaled_cr = layer.unscale(unscaled_cr, force_process=True)

In [19]:
outputs = instantiate(cfg.task.outputs)
for output in outputs:
    output.to("cuda")

In [20]:
np_result = trainer.model(np_test)
cr_result = model(cr_test)

In [21]:
loss, loss_contrib = trainer.loss(unscaled_np, np_result)

In [22]:
loss

tensor(0.5325, device='cuda:0', grad_fn=<AddBackward0>)

In [23]:
loss_contrib

{'forces': tensor(0.4559, device='cuda:0', grad_fn=<MeanBackward0>),
 'total_energy': tensor(0.0766, device='cuda:0', grad_fn=<MeanBackward0>)}

In [24]:
outputs[0].calculate_loss(unscaled_cr, cr_result, return_num_obs=False).detach()

tensor(0.0766, device='cuda:0')

In [25]:
outputs[1].calculate_loss(unscaled_cr, cr_result, return_num_obs=False).detach()

tensor(0.4559, device='cuda:0')

In [42]:
unscaled_cr['energy']

tensor([-0.6850, -0.6627, -0.6082, -0.5363, -0.4645], device='cuda:0')

In [46]:
np_result['total_energy']

tensor([[-406750.6875],
        [-406750.6562],
        [-406750.6562],
        [-406750.6250],
        [-406750.6250]], device='cuda:0', grad_fn=<AddBackward0>)

In [43]:
unscaled_np['total_energy']

tensor([[-0.6850],
        [-0.6627],
        [-0.6082],
        [-0.5363],
        [-0.4645]], device='cuda:0')

In [115]:
trainer

AttributeError: 'Trainer' object has no attribute 'optimizer'

In [89]:
unscaled_cr['energy']

tensor([-0.7405], device='cuda:0')

In [91]:
unscaled_np['total_energy']

tensor([-0.6227], device='cuda:0')

In [92]:
unscaled_np['forces']

tensor([[-7.2516e-02,  4.0498e-02, -8.9127e-02],
        [-2.2105e-01,  1.3086e-01, -1.7203e-01],
        [ 4.1190e-01,  7.1826e-02,  1.2072e-01],
        [ 1.7269e-01,  1.0772e-01, -2.6124e-02],
        [ 5.0346e-01,  2.4314e-01, -8.9178e-01],
        [ 6.7509e-01, -8.5078e-01,  2.5630e-01],
        [ 6.9867e-01, -9.9943e-02,  4.0456e-01],
        [-8.6034e-01, -5.1414e-01, -1.3998e-01],
        [ 2.6646e-01,  1.4698e-01,  9.8090e-01],
        [ 2.2210e+00,  4.7495e-01,  5.1323e-01],
        [-1.4480e+00, -3.6098e-01, -3.0402e-01],
        [-7.6281e-01,  9.2996e-02,  1.5675e+00],
        [-8.9484e-01,  3.6686e-01, -1.9045e+00],
        [-8.0178e-01,  1.9299e-01, -2.3867e-01],
        [ 5.5745e-02, -3.0974e-01,  6.0622e-02],
        [-2.7871e-01,  2.0410e-01, -6.5777e-02],
        [ 3.3964e-01,  2.4596e-02,  9.7423e-02],
        [ 1.3444e-01,  3.2541e-01, -1.0568e-04],
        [ 1.5653e-02, -2.4224e-01, -4.4820e-02],
        [ 8.2317e-02,  3.9407e-02,  1.5965e-01],
        [-2.3704e-01

In [96]:
cr_rescale[0].scale_by

tensor([28.5284], device='cuda:0')

In [97]:
np_rescale[0].scale_by

tensor(25.2939, device='cuda:0')

In [103]:
cr_model.output_modules[2].scale_by

tensor([25.2939], device='cuda:0')

In [93]:
unscaled_cr['energy']

tensor([-0.7405], device='cuda:0')

In [94]:
unscaled_cr['forces']

tensor([[-6.4295e-02,  3.5906e-02, -7.9022e-02],
        [-1.9599e-01,  1.1602e-01, -1.5252e-01],
        [ 3.6520e-01,  6.3683e-02,  1.0703e-01],
        [ 1.5311e-01,  9.5507e-02, -2.3162e-02],
        [ 4.4638e-01,  2.1558e-01, -7.9067e-01],
        [ 5.9855e-01, -7.5432e-01,  2.2724e-01],
        [ 6.1946e-01, -8.8612e-02,  3.5869e-01],
        [-7.6280e-01, -4.5585e-01, -1.2411e-01],
        [ 2.3625e-01,  1.3031e-01,  8.6969e-01],
        [ 1.9692e+00,  4.2110e-01,  4.5504e-01],
        [-1.2838e+00, -3.2006e-01, -2.6955e-01],
        [-6.7632e-01,  8.2452e-02,  1.3897e+00],
        [-7.9338e-01,  3.2527e-01, -1.6886e+00],
        [-7.1087e-01,  1.7111e-01, -2.1161e-01],
        [ 4.9425e-02, -2.7462e-01,  5.3749e-02],
        [-2.4711e-01,  1.8096e-01, -5.8320e-02],
        [ 3.0113e-01,  2.1807e-02,  8.6378e-02],
        [ 1.1920e-01,  2.8852e-01, -9.3697e-05],
        [ 1.3878e-02, -2.1478e-01, -3.9738e-02],
        [ 7.2984e-02,  3.4939e-02,  1.4155e-01],
        [-2.1017e-01

In [80]:
import copy

In [81]:
cr_loss = copy.copy(trainer.loss)

In [108]:
cr_loss.keys = ['forces', 'total_energy']
cr_loss.coeffs['total_energy'] =  cr_loss.coeffs.pop('energy')
cr_loss.funcs['total_energy'] = cr_loss.funcs.pop('energy')

In [88]:
cr_loss(unscaled_cr, cr_result)

(tensor(0.4475, device='cuda:0', grad_fn=<AddBackward0>),
 {'forces': tensor(0.2772, device='cuda:0', grad_fn=<MeanBackward0>),
  'energy': tensor(0.1703, device='cuda:0', grad_fn=<MeanBackward0>)})

In [34]:
cr_result = cr_model(cr_test)
np_result = trainer.model(np_test)

In [35]:
cr_result

{'forces': tensor([[-1.8028e-02,  4.5311e-03, -5.5574e-03],
         [ 1.7407e-02, -9.1300e-03,  4.3322e-03],
         [-1.9957e-02, -6.1250e-04, -5.3755e-03],
         [-1.1042e-02, -1.9254e-02, -4.7222e-04],
         [-1.0571e-02, -3.4560e-03, -7.7775e-03],
         [ 3.3908e-02,  3.6720e-02,  1.9644e-03],
         [-1.6152e-02, -1.3114e-02,  6.4452e-03],
         [-4.1041e-02, -1.3427e-02, -1.1256e-02],
         [ 1.8837e-02,  1.6667e-02,  5.5576e-02],
         [ 1.2808e-02,  5.7524e-04,  3.4341e-03],
         [ 2.9525e-02, -2.2228e-03,  8.6335e-03],
         [ 3.3773e-02, -6.0120e-03, -3.9171e-02],
         [ 2.7731e-03, -1.2917e-03, -4.0038e-03],
         [-7.9645e-03,  1.9352e-02, -4.7437e-03],
         [ 9.1247e-03, -8.4660e-03,  3.6157e-03],
         [-1.8233e-04,  4.9967e-03,  8.4954e-05],
         [ 2.9934e-03, -3.1448e-04,  8.4308e-04],
         [ 9.3691e-04,  2.7873e-03, -1.1192e-04],
         [-2.1421e-02, -4.3765e-02,  2.1222e-03],
         [-1.2655e-02,  2.1952e-02,  3.0

In [55]:
cr_model.representation.embeddings.chemical_embedding.load_state_dict(trainer.model.model.func.chemical_embedding.state_dict())
cr_model.representation.interactions[0].load_state_dict(trainer.model.model.func.layer0_convnet.state_dict(), strict=False)
cr_model.representation.interactions[1].load_state_dict(trainer.model.model.func.layer1_convnet.state_dict(), strict=False)
cr_model.representation.interactions[2].load_state_dict(trainer.model.model.func.layer2_convnet.state_dict(), strict=False)
cr_model.representation.interactions[3].load_state_dict(trainer.model.model.func.layer3_convnet.state_dict(), strict=False)

cr_model.representation.interactions[0].conv.avg_num_neighbors[0] = trainer.model.model.func.layer0_convnet.conv.avg_num_neighbors
cr_model.representation.interactions[1].conv.avg_num_neighbors[0] = trainer.model.model.func.layer1_convnet.conv.avg_num_neighbors
cr_model.representation.interactions[2].conv.avg_num_neighbors[0] = trainer.model.model.func.layer2_convnet.conv.avg_num_neighbors
cr_model.representation.interactions[3].conv.avg_num_neighbors[0] = trainer.model.model.func.layer3_convnet.conv.avg_num_neighbors

cr_model.representation.readout_mlp[0].load_state_dict(trainer.model.model.func.conv_to_output_hidden.linear.state_dict())
cr_model.representation.readout_mlp[1].load_state_dict(trainer.model.model.func.output_hidden_to_scalar.linear.state_dict())

cr_model.output_modules[2].scale_by[0] = trainer.model.scale_by
cr_model.output_modules[2].shift_by = trainer.model.shift_by

In [10]:
cfg = read_user_config("nequip.yaml", config_path="../curator/configs")
OmegaConf.save(cfg, 'config.yaml', resolve=True)

data = instantiate(cfg.data)
cr_model = instantiate(cfg.model)
task = instantiate(cfg.task, model=cr_model)
cr_trainer = instantiate(cfg.trainer)
task.save_configuration(cfg)
# set up logger
console_logger = logging.getLogger("console")
console_logger.setLevel(logging.DEBUG)
fh = logging.FileHandler(os.path.join(cfg.run_path, "test_training.log"), mode="w")
fh.setFormatter(logging.Formatter("%(message)s"))
sh = logging.StreamHandler()
sh.setFormatter(logging.Formatter("%(message)s"))

data.setup()
console_logger.addHandler(fh)
console_logger.addHandler(sh)
console_logger.info("Initiating console logger")

/home/xinyang/miniconda3/lib/python3.11/site-packages/torch/jit/_check.py:172: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_test_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
Initiating console logger
Initiating console logger


In [6]:
cr_trainer.fit(task, data)

/home/xinyang/miniconda3/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /home/xinyang/xin_test/Curator/gnn-active_learning/model_path exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                   | Params
---------------------------------------------------
0 | model   | NeuralNetworkPotential | 392 K 
1 | outputs | ModuleList             | 0     
---------------------------------------------------
392 K     Trainable params
0         Non-trainable params
392 K     Total params
1.571     Total estimated model params size (MB)
/home/xinyang/miniconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argume

Epoch 3:   5%|▌         | 1/20 [00:00<00:06,  2.84it/s, v_num=3, train_total_loss_step=9.81e+5, train_energy_loss_step=7.47e+5, train_forces_loss_step=2.34e+5, val_total_loss_step=1.65e+11, val_energy_loss_step=1.65e+11, val_forces_loss_step=545.0, val_total_loss_epoch=1.65e+11, val_energy_loss_epoch=1.65e+11, val_forces_loss_epoch=618.0, train_total_loss_epoch=1.6e+6, train_energy_loss_epoch=1.28e+6, train_forces_loss_epoch=3.16e+5]     

RuntimeError: DataLoader worker (pid 15403) is killed by signal: Aborted. 

In [15]:
next(iter(data.train_dataloader()))['_n_atoms']

tensor([21, 21, 21, 21, 21])

In [4]:
trainer.fit(task, data)

/home/xinyang/miniconda3/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /home/xinyang/xin_test/Curator/gnn-active_learning/model_path exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                   | Params
---------------------------------------------------
0 | model   | NeuralNetworkPotential | 1.1 M 
1 | outputs | ModuleList             | 0     
---------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.224     Total estimated model params size (MB)
/home/xinyang/miniconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argume

Training: 0it [00:00, ?it/s]



Start training model


Epoch 0:   0%|          | 0/200 [00:00<?, ?it/s] 



Training
# epoch      batch      total_loss     energy_loss     forces_loss      energy_mae     energy_rmse      forces_mae     forces_rmse
      0          0       68717.312        7225.017       61492.297          37.944          38.013          19.216          25.442


...

     25         80         458.407           5.703         452.704           0.825           1.068           1.607           2.183


Epoch 25:  45%|████▌     | 90/200 [00:20<00:25,  4.32it/s, v_num=32, train_total_loss_step=425.0, train_energy_loss_step=35.40, train_forces_loss_step=390.0, val_total_loss_step=745.0, val_energy_loss_step=5.300, val_forces_loss_step=740.0, val_total_loss_epoch=700.0, val_energy_loss_epoch=6.280, val_forces_loss_epoch=694.0, train_total_loss_epoch=491.0, train_energy_loss_epoch=56.60, train_forces_loss_epoch=435.0]

     25         90         497.379         137.696         359.683           5.169           5.248           1.405           1.946


Epoch 25:  48%|████▊     | 96/200 [00:22<00:24,  4.31it/s, v_num=32, train_total_loss_step=463.0, train_energy_loss_step=59.50, train_forces_loss_step=404.0, val_total_loss_step=745.0, val_energy_loss_step=5.300, val_forces_loss_step=740.0, val_total_loss_epoch=700.0, val_energy_loss_epoch=6.280, val_forces_loss_epoch=694.0, train_total_loss_epoch=491.0, train_energy_loss_epoch=56.60, train_forces_loss_epoch=435.0]

/home/xinyang/miniconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [5]:
state_dict = torch.load('model_path/best_model_epoch=0-step=200-val_total_loss=4915.70.ckpt')

In [15]:
state_dict['outputs'][0].train_metrics['mae'].compute()

tensor(19.0259, device='cuda:0')

In [6]:
trainer._results

In [4]:
from mace.calculators import mace_mp

In [5]:
mace_calc = mace_mp(model='large')
mace_model = mace_calc.models[0]

Using Materials Project MACE for MACECalculator with /home/xinyang/.cache/mace/43hjdekn
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Default dtype float32 does not match model dtype float64, converting models to float32.


In [6]:
# load mace foundation model

for readout in mace_model.readouts:
    if readout.__class__.__name__ == 'NonLinearReadoutBlock':
        MLP_irreps = readout.hidden_irreps
        gate = readout.non_linearity.acts[0]
        break

num_interactions = len(mace_model.interactions)
curator_mace = curator.model.MACE(
    cutoff=float(mace_model.r_max),
    num_interactions=num_interactions,
    correlation=[prod.symmetric_contractions.contractions[0].correlation for prod in mace_model.products],
    species=[chemical_symbols[i] for i in mace_model.atomic_numbers],
    num_elements=len(mace_model.atomic_numbers),
    hidden_irreps=mace_model.interactions[0].hidden_irreps,
    edge_sh_irreps=mace_model.spherical_harmonics.irreps_out,
    # node_irreps: Union[o3.Irreps, str, None] = None,
    MLP_irreps=MLP_irreps,
    avg_num_neighbors=mace_model.interactions[0].avg_num_neighbors,
    num_basis=len(mace_model.radial_embedding.bessel_fn.bessel_weights),
    power=int(mace_model.radial_embedding.cutoff_fn.p),
    gate=gate,
)

In [8]:
mace_model.interactions[0].avg_num_neighbors

61.964672446250916

In [11]:
curator_mace.interactions[0].avg_num_neighbors

tensor([61.9647])

In [11]:
curator_mace.embeddings.chemical_embedding.load_state_dict(mace_model.node_embedding.state_dict())
for i in range(num_interactions):
    # load interaction layers
    avg = curator_mace.interactions[i].avg_num_neighbors
    curator_mace.interactions[i].avg_num_neighbors = torch.tensor([mace_model.interactions[i].avg_num_neighbors], dtype=avg.dtype, device=avg.device)
    curator_mace.interactions[i].linear_1.load_state_dict(mace_model.interactions[i].linear_up.state_dict())
    curator_mace.interactions[i].linear_2.load_state_dict(mace_model.interactions[i].linear.state_dict())

    # load product layers
    curator_mace.products[i].load_state_dict(mace_model.products[i].state_dict())
    
    if mace_model.readouts[i].__class__.__name__ == 'NonLinearReadoutBlock':
        curator_mace.readouts[i].load_state_dict(mace_model.readouts[i].state_dict())
    else:
        curator_mace.readouts[i].load_state_dict(mace_model.readouts[i].linear.state_dict())

In [17]:
input_modules = [curator.layer.PairwiseDistance()]
output_modules = [curator.layer.AtomwiseReduce()]

In [19]:
atoms = read('/home/xinyang/Au-water_test.traj', -1)

In [20]:
atoms

Atoms(symbols='H113Au64O59', pbc=True, cell=[11.5889, 11.5889, 35.0], momenta=..., calculator=SinglePointCalculator(...))

In [21]:
mace.data.config_from_atoms(atoms)

Configuration(atomic_numbers=array([79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79,
       79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79,
       79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79,
       79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 

In [15]:
mace_model.scale_shift.shift

tensor(0.1641, device='cuda:0')

In [9]:
curator_mace.readouts

ModuleList(
  (0): Linear(128x0e+128x1o+128x2e -> 1x0e | 128 weights)
  (1): AtomwiseNonLinear(
    (linear_1): Linear(128x0e -> 16x0e | 2048 weights)
    (non_linearity): Activation [x] (16x0e -> 16x0e)
    (linear_2): Linear(16x0e -> 1x0e | 16 weights)
  )
)

In [8]:
mace_model.readouts

ModuleList(
  (0): LinearReadoutBlock(
    (linear): Linear(128x0e+128x1o+128x2e -> 1x0e | 128 weights)
  )
  (1): NonLinearReadoutBlock(
    (linear_1): Linear(128x0e -> 16x0e | 2048 weights)
    (non_linearity): Activation [x] (16x0e -> 16x0e)
    (linear_2): Linear(16x0e -> 1x0e | 16 weights)
  )
)

AttributeError: 'MACE' object has no attribute 'readouts'

In [15]:
mace_model.products

ModuleList(
  (0): EquivariantProductBasisBlock(
    (symmetric_contractions): SymmetricContraction(
      (contractions): ModuleList(
        (0): Contraction(
          (contractions_weighting): ModuleList(
            (0-1): 2 x GraphModule()
          )
          (contractions_features): ModuleList(
            (0-1): 2 x GraphModule()
          )
          (weights): ParameterList(
              (0): Parameter containing: [torch.float32 of size 89x4x128 (GPU 0)]
              (1): Parameter containing: [torch.float32 of size 89x1x128 (GPU 0)]
          )
          (graph_opt_main): GraphModule()
        )
        (1): Contraction(
          (contractions_weighting): ModuleList(
            (0-1): 2 x GraphModule()
          )
          (contractions_features): ModuleList(
            (0-1): 2 x GraphModule()
          )
          (weights): ParameterList(
              (0): Parameter containing: [torch.float32 of size 89x6x128 (GPU 0)]
              (1): Parameter containing: [tor

In [7]:
curator_mace

MACE(
  (embeddings): ModuleDict(
    (onehot_embedding): OneHotAtomEncoding(
      (type_mapper): TypeMapper()
    )
    (radial_basis): RadialBasisEdgeEncoding(
      (basis): BesselBasis()
      (cutoff_fn): PolynomialCutoff()
    )
    (sphere_harmonics): SphericalHarmonicEdgeAttrs(
      (sh): SphericalHarmonics()
    )
    (chemical_embedding): AtomwiseLinear(
      (linear): Linear(89x0e -> 128x0e | 11392 weights)
    )
  )
  (interactions): ModuleList(
    (0): RealAgnosticResidualInteractionBlock(
      (linear_1): Linear(128x0e -> 128x0e | 16384 weights)
      (conv_tp): TensorProduct(128x0e x 1x0e+1x1o+1x2e+1x3o -> 128x0e+128x1o+128x2e+128x3o | 512 paths | 512 weights)
      (conv_tp_weights): FullyConnectedNet[10, 64, 64, 64, 512]
      (linear_2): Linear(128x0e+128x1o+128x2e+128x3o -> 128x0e+128x1o+128x2e+128x3o | 65536 weights)
      (skip_tp): FullyConnectedTensorProduct(128x0e x 89x0e -> 128x0e+128x1o+128x2e | 1458176 paths | 1458176 weights)
      (reshape): reshape_ir

In [12]:
curator_mace.interactions[i]

RealAgnosticResidualInteractionBlock(
  (linear_1): Linear(128x0e -> 128x0e | 16384 weights)
  (conv_tp): TensorProduct(128x0e x 1x0e+1x1o+1x2e+1x3o -> 128x0e+128x1o+128x2e+128x3o | 512 paths | 512 weights)
  (conv_tp_weights): FullyConnectedNet[10, 64, 64, 64, 512]
  (linear_2): Linear(128x0e+128x1o+128x2e+128x3o -> 128x0e+128x1o+128x2e+128x3o | 65536 weights)
  (skip_tp): FullyConnectedTensorProduct(128x0e x 89x0e -> 128x0e+128x1o+128x2e | 1458176 paths | 1458176 weights)
  (reshape): reshape_irreps()
)

In [13]:
mace_model.interactions[i]

RealAgnosticResidualInteractionBlock(
  (linear_up): Linear(128x0e -> 128x0e | 16384 weights)
  (conv_tp): TensorProduct(128x0e x 1x0e+1x1o+1x2e+1x3o -> 128x0e+128x1o+128x2e+128x3o | 512 paths | 512 weights)
  (conv_tp_weights): FullyConnectedNet[10, 64, 64, 64, 512]
  (linear): Linear(128x0e+128x1o+128x2e+128x3o -> 128x0e+128x1o+128x2e+128x3o | 65536 weights)
  (skip_tp): FullyConnectedTensorProduct(128x0e x 89x0e -> 128x0e+128x1o+128x2e | 1458176 paths | 1458176 weights)
  (reshape): reshape_irreps()
)

RuntimeError: Error(s) in loading state_dict for RealAgnosticResidualInteractionBlock:
	Missing key(s) in state_dict: "avg_num_neighbors", "linear_1.weight", "linear_1.bias", "linear_1.output_mask", "linear_2.weight", "linear_2.bias", "linear_2.output_mask". 
	Unexpected key(s) in state_dict: "linear_up.weight", "linear_up.bias", "linear_up.output_mask", "linear.weight", "linear.bias", "linear.output_mask". 

In [8]:
mace_model

ScaleShiftMACE(
  (node_embedding): LinearNodeEmbeddingBlock(
    (linear): Linear(89x0e -> 128x0e | 11392 weights)
  )
  (radial_embedding): RadialEmbeddingBlock(
    (bessel_fn): BesselBasis(r_max=6.0, num_basis=10, trainable=False)
    (cutoff_fn): PolynomialCutoff(p=5.0, r_max=6.0)
  )
  (spherical_harmonics): SphericalHarmonics()
  (atomic_energies_fn): AtomicEnergiesBlock(energies=[-3.6672, -1.3321, -3.4821, -4.7367, -7.7249, -8.4056, -7.3601, -7.2846, -4.8965, 0.0000, -2.7594, -2.8140, -4.8469, -7.6948, -6.9633, -4.6726, -2.8117, -0.0626, -2.6176, -5.3905, -7.8858, -10.2684, -8.6651, -9.2331, -8.3050, -7.0490, -5.5774, -5.1727, -3.2521, -1.2902, -3.5271, -4.7085, -3.9765, -3.8862, -2.5185, 6.7669, -2.5635, -4.9380, -10.1498, -11.8469, -12.1389, -8.7917, -8.7869, -7.7809, -6.8500, -4.8910, -2.0634, -0.6396, -2.7887, -3.8186, -3.5871, -2.8804, -1.6356, 9.8467, -2.7653, -4.9910, -8.9337, -8.7356, -8.0190, -8.2515, -7.5917, -8.1697, -13.5927, -18.5175, -7.6474, -8.1230, -7.6078, -6.

In [107]:
curator_mace

MACE(
  (embeddings): ModuleDict(
    (onehot_embedding): OneHotAtomEncoding(
      (type_mapper): TypeMapper()
    )
    (radial_basis): RadialBasisEdgeEncoding(
      (basis): BesselBasis()
      (cutoff_fn): PolynomialCutoff()
    )
    (sphere_harmonics): SphericalHarmonicEdgeAttrs(
      (sh): SphericalHarmonics()
    )
    (chemical_embedding): AtomwiseLinear(
      (linear): Linear(89x0e -> 128x0e | 11392 weights)
    )
  )
  (interactions): ModuleList(
    (0): RealAgnosticResidualInteractionBlock(
      (linear_1): Linear(128x0e -> 128x0e | 16384 weights)
      (conv_tp): TensorProduct(128x0e x 1x0e+1x1o+1x2e+1x3o -> 128x0e+128x1o+128x2e+128x3o | 512 paths | 512 weights)
      (conv_tp_weights): FullyConnectedNet[10, 64, 64, 64, 512]
      (linear_2): Linear(128x0e+128x1o+128x2e+128x3o -> 128x0e+128x1o+128x2e+128x3o | 65536 weights)
      (skip_tp): FullyConnectedTensorProduct(128x0e x 89x0e -> 128x0e+128x1o+128x2e | 1458176 paths | 1458176 weights)
      (reshape): reshape_ir

In [110]:
sum(p.numel() for p in mace_model.parameters())

5725072

In [111]:
sum(p.numel() for p in curator_mace.parameters())

5725082

In [106]:
mace_model

ScaleShiftMACE(
  (node_embedding): LinearNodeEmbeddingBlock(
    (linear): Linear(89x0e -> 128x0e | 11392 weights)
  )
  (radial_embedding): RadialEmbeddingBlock(
    (bessel_fn): BesselBasis(r_max=6.0, num_basis=10, trainable=False)
    (cutoff_fn): PolynomialCutoff(p=5.0, r_max=6.0)
  )
  (spherical_harmonics): SphericalHarmonics()
  (atomic_energies_fn): AtomicEnergiesBlock(energies=[-3.6672, -1.3321, -3.4821, -4.7367, -7.7249, -8.4056, -7.3601, -7.2846, -4.8965, 0.0000, -2.7594, -2.8140, -4.8469, -7.6948, -6.9633, -4.6726, -2.8117, -0.0626, -2.6176, -5.3905, -7.8858, -10.2684, -8.6651, -9.2331, -8.3050, -7.0490, -5.5774, -5.1727, -3.2521, -1.2902, -3.5271, -4.7085, -3.9765, -3.8862, -2.5185, 6.7669, -2.5635, -4.9380, -10.1498, -11.8469, -12.1389, -8.7917, -8.7869, -7.7809, -6.8500, -4.8910, -2.0634, -0.6396, -2.7887, -3.8186, -3.5871, -2.8804, -1.6356, 9.8467, -2.7653, -4.9910, -8.9337, -8.7356, -8.0190, -8.2515, -7.5917, -8.1697, -13.5927, -18.5175, -7.6474, -8.1230, -7.6078, -6.

In [15]:
model.representation.products[0].symmetric_contractions.contractions[0]

Contraction(
  (contractions_weighting): ModuleList(
    (0-1): 2 x GraphModule()
  )
  (contractions_features): ModuleList(
    (0-1): 2 x GraphModule()
  )
  (weights): ParameterList(
      (0): Parameter containing: [torch.float32 of size 3x4x128]
      (1): Parameter containing: [torch.float32 of size 3x1x128]
  )
  (graph_opt_main): GraphModule()
)

In [57]:
from curator.layer._symmetric_contraction import Contraction, SymmetricContraction

In [103]:
from curator.layer import (
    OneHotAtomEncoding,
    AtomwiseLinear,
    AtomwiseNonLinear,
    RadialBasisEdgeEncoding,
    SphericalHarmonicEdgeAttrs,
    RealAgnosticResidualInteractionBlock,
    EquivariantProductBasisBlock,
)

cutoff=float(mace_model.r_max)
num_interactions=len(mace_model.interactions)
correlation=3
species=[chemical_symbols[i] for i in mace_model.atomic_numbers]
num_elements=len(mace_model.atomic_numbers)
hidden_irreps=mace_model.interactions[0].hidden_irreps
edge_sh_irreps=mace_model.spherical_harmonics.irreps_out
node_irreps=None
MLP_irreps=MLP_irreps
avg_num_neighbors=mace_model.interactions[0].avg_num_neighbors
num_basis=len(mace_model.radial_embedding.bessel_fn.bessel_weights)
power=int(mace_model.radial_embedding.cutoff_fn.p)
gate=gate

if num_elements is None:
    num_elements = len(species)

# hidden feature irreps
if hidden_irreps is not None:
    hidden_irreps = o3.Irreps(hidden_irreps) if isinstance(hidden_irreps, str) else hidden_irreps
else:
    hidden_irreps = o3.Irreps(
        [
            (num_features, (l, p))
            for p in ((1, -1) if parity else (1,))
            for l in range(lmax + 1)
        ]
    )
# MACE prohibits some irreps like 0e, 1e to be used
forbidden_ir = ['0o', '1e', '2o', '3e', '4o']
hidden_irreps = o3.Irreps([irrep for irrep in hidden_irreps if str(irrep.ir) not in forbidden_ir])
num_features = hidden_irreps.count(o3.Irrep(0, 1))

## handling irreps
# chemical embedding irreps
if node_irreps is None:
    node_irreps = o3.Irreps([(num_features, (0, 1))])
elif isinstance(node_irreps, str):
    node_irreps = o3.Irreps(node_irreps)
else:
    node_irreps = node_irreps
# edge sphere harmonic irreps
if edge_sh_irreps is None:
    edge_sh_irreps = o3.Irreps.spherical_harmonics(lmax, p=-1 if parity else 1)
elif isinstance(edge_sh_irreps, str):
    edge_sh_irreps = o3.Irreps(edge_sh_irreps)
else:
    edge_sh_irreps = edge_sh_irreps

# MLP_irreps
if MLP_irreps is None:
    MLP_irreps = o3.Irreps([(max(1, num_features // 2), (0, 1))])
elif isinstance(MLP_irreps, str):
    MLP_irreps = o3.Irreps(MLP_irreps)
else:
    MLP_irreps = MLP_irreps
    
embeddings = nn.ModuleDict()
embeddings['onehot_embedding'] = OneHotAtomEncoding(num_elements=num_elements, species=species)
embeddings['radial_basis'] = RadialBasisEdgeEncoding(
    cutoff=cutoff,
    basis_kwargs={'num_basis': num_basis},
    cutoff_kwargs={'power': power},
)
embeddings['sphere_harmonics'] = SphericalHarmonicEdgeAttrs(edge_sh_irreps=edge_sh_irreps)

irreps_in = {
    properties.edge_diff_embedding: embeddings.sphere_harmonics.irreps_out,
    properties.edge_dist_embedding: embeddings.radial_basis.irreps_out,
}
irreps_in.update(embeddings.onehot_embedding.irreps_out)

embeddings['chemical_embedding'] = AtomwiseLinear(
    irreps_in=irreps_in[properties.node_attr],
    irreps_out=node_irreps,
)
irreps_in[properties.node_feat] = embeddings.chemical_embedding.irreps_out

interaction_irreps = (edge_sh_irreps * num_features).sort()[0].simplify()

interactions = torch.nn.ModuleList()
products = torch.nn.ModuleList()
readout_mlp = torch.nn.ModuleList()
gate_fn = activation_fn[gate] if isinstance(gate, str) else gate
# interaction blocks
for i in range(num_interactions):
    hidden_irreps_out = str(hidden_irreps[0]) if i == num_interactions - 1 else hidden_irreps
    if i > 0:
        irreps_in[properties.node_feat] = hidden_irreps
    inter = RealAgnosticResidualInteractionBlock(
        irreps_in=irreps_in,
        target_irreps=interaction_irreps,
        hidden_irreps=hidden_irreps_out,
        avg_num_neighbors=avg_num_neighbors,
    )
    interactions.append(inter)
    
    prod = EquivariantProductBasisBlock(
        node_feats_irreps=inter.target_irreps if i == 0 else interaction_irreps,
        target_irreps=hidden_irreps_out,
        correlation=correlation,
        num_elements=num_elements,
        use_sc=True,
    )
    products.append(prod)
    
    if i == num_interactions - 1:
        readout = AtomwiseNonLinear(
            irreps_in=hidden_irreps_out, 
            MLP_irreps=MLP_irreps,
            gate=gate_fn,
        )
    else:
        readout = o3.Linear(irreps_in=hidden_irreps_out, irreps_out=o3.Irreps('1x0e'))
    readout_mlp.append(readout)

/home/xinyang/miniconda3/lib/python3.11/site-packages/torch/jit/_check.py:172: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


In [5]:
cfg = read_user_config("user_cfg.yaml", config_path="../curator/configs")
OmegaConf.save(cfg, 'config.yaml', resolve=True)

data = instantiate(cfg.data)
model = instantiate(cfg.model)
task = instantiate(cfg.task, model=model)
trainer = instantiate(cfg.trainer)
task.save_configuration(cfg)
# set up logger
console_logger = logging.getLogger("console")
console_logger.setLevel(logging.DEBUG)
fh = logging.FileHandler(os.path.join(cfg.run_path, "training.log"), mode="w")
fh.setFormatter(logging.Formatter("%(message)s"))
sh = logging.StreamHandler()
sh.setFormatter(logging.Formatter("%(message)s"))

console_logger.addHandler(fh)
console_logger.addHandler(sh)
console_logger.info("Initiating console logger")

trainer.fit(task, data)

/home/xinyang/miniconda3/lib/python3.11/site-packages/torch/jit/_check.py:172: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_test_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
Initiating console logger


/home/xinyang/miniconda3/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /home/xinyang/xin_test/Curator/gnn-active_learning/model_path exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                   | Params
---------------------------------------------------
0 | model   | NeuralNetworkPotential | 1.1 M 
1 | outputs | ModuleList             | 0     
---------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.224     Total estimated model params size (MB)
/home/xinyang/miniconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argume

Training: 0it [00:00, ?it/s]



Start training model


Epoch 0:   0%|          | 0/200 [00:00<?, ?it/s] 



Training
    0          0      269794.000      196575.453       73218.531         197.969         198.280          20.779          27.762
training_step.train_loss: 269794.000      training_step.energy_loss: 196575.453      training_step.forces_loss: 73218.531       training_step.train_energy_mae: 197.969         training_step.train_energy_rmse: 198.280         training_step.train_forces_mae: 20.779          training_step.train_forces_rmse: 27.762          


...

Epoch 0:  95%|█████████▌| 190/200 [00:55<00:02,  3.43it/s, v_num=8, train_loss=7.17e+3, energy_loss=721.0, forces_loss=6.45e+3]

    0        190        6222.728         110.071        6112.656           3.419           4.692           5.862           8.021
training_step.train_loss: 6222.728        training_step.energy_loss: 110.071         training_step.forces_loss: 6112.656        training_step.train_energy_mae: 3.419           training_step.train_energy_rmse: 4.692           training_step.train_forces_mae: 5.862           training_step.train_forces_rmse: 8.021           


Epoch 0: 100%|██████████| 200/200 [00:57<00:00,  3.48it/s, v_num=8, train_loss=4.46e+3, energy_loss=111.0, forces_loss=4.35e+3]



Validating
# epoch      batch      total_loss     energy_loss     forces_loss      energy_mae     energy_rmse      forces_mae     forces_rmse


TypeError: unsupported format string passed to MeanAbsoluteError.__format__

In [9]:
task.batch_size

AttributeError: 'LitNNP' object has no attribute 'batch_size'

In [11]:
task.outputs[0].metrics['train']

ModuleDict(
  (mae): MeanAbsoluteError()
  (rmse): MeanSquaredError()
)

In [13]:
isinstance(task.outputs[0].metrics['train'], Dict[str, Metric])

TypeError: Subscripted generics cannot be used with class and instance checks

In [ ]:
nn.ModuleDict

In [ ]:
a = {'a': 1, 'b': 2}

In [ ]:
a.get('c', 3)

In [ ]:
a

In [ ]:
for k in a:
    print(a[k])

In [ ]:
cfg = read_user_config('nequip.yaml')

In [ ]:
import logging
logger = logging.getLogger("simple_example")
logger.setLevel(logging.DEBUG)
# 建立一个filehandler来把日志记录在文件里，级别为debug以上
fh = logging.FileHandler("spam.log")
fh.setLevel(logging.DEBUG)
# 建立一个streamhandler来把日志打在CMD窗口上，级别为error以上
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
# 设置日志格式
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
ch.setFormatter(formatter)
fh.setFormatter(formatter)
#将相应的handler添加在logger对象中
logger.addHandler(ch)
# logger.addHandler(fh)
# 开始打日志
logger.debug("debug message")
logger.info("info message")
logger.warn("warn message")
logger.error("error message")
logger.critical("critical message")

In [ ]:
model = instantiate(cfg.model)

In [ ]:
script(model)

In [ ]:
cfg.new = 'new'

In [ ]:
config

In [ ]:
config.new = 'new'

In [ ]:
cfg.model

In [ ]:
model_path = [str(f) for f in Path(f"/home/xinyang").rglob("best_model_epoch*")]
val_loss = float('inf')
index = 0
for i, p in enumerate(model_path):
    loss = float(p.split('=')[-1].rstrip('.ckpt'))
    if loss < val_loss:
        val_loss = loss
        index = i
model_path = model_path[index]  
log.info(f"Deploy trained model from {model_path}")
task = task.load_from_checkpoint(checkpoint_path=f"{model_path}", model=model)
model_compiled = script(task.model)
metadata = {
    "cutoff": str(model_compiled.representation.cutoff).encode("ascii"),
    
}
model_compiled.save(f"{config.run_path}/compiled_model.pt", _extra_files=metadata)
log.info(f"Deploying compiled model at <{config.run_path}/compiled_model.pt>")

In [ ]:
for name, param in model.named_parameters():
    if isinstance(param, float):
        print(name)

In [ ]:
for k, v in model.named_modules():
    print(k)

In [ ]:
module_dict = dict(model.named_modules())

In [ ]:
model.representation.interaction[0]

In [ ]:
torch.zeros((1,))

In [ ]:
for name, param in module_dict['representation.interactions.0.conv'].named_buffers():
    print(name, param)

In [ ]:
module_dict['representation.interactions.0.conv'].avg_num_neighbors

In [ ]:
import e3nn

In [ ]:
new_dict = dict(module_dict['representation.interactions.0.conv'].named_modules())

In [ ]:
new_dict.keys()

In [ ]:
e3nn.util.jit.script(module_dict['representation.interactions.0.conv.linear_1'])

In [ ]:
e3nn.util.jit.script(module_dict['representation.interactions.0.conv'])

In [ ]:
for modules in model.
script(model)

In [ ]:
for name, param in model.named_buffers():
    print(name, type(param))

In [ ]:
model_path = [str(f) for f in Path(f"/home/xinyang").glob("best_model_epoch*")]
val_loss = float('inf')
index = 0
for i, p in enumerate(model_path):
    loss = float(p.split('=')[-1].rstrip('.ckpt'))
    if loss < val_loss:
        val_loss = loss
        index = i
model_path = model_path[index]

task = task.load_from_checkpoint(model_path, model=model, strict=False)
# model_compiled = script(task.model)

# metadata = {
#     "cutoff": str(model_compiled.representation.cutoff).encode("ascii"),
#     "model_config": cfg.model,
# }

# model_compiled.save('/home/xinyang/compiled_model.pt', _extra_files=metadata)

In [ ]:
cfg = read_user_config('nequip.yaml')

In [ ]:
model = instantiate(cfg.model)

In [ ]:
script(model)

In [ ]:
model.representation.embeddings.onehot_embedding.species

In [ ]:
task = instantiate(cfg.task, model=model)

In [ ]:
module_dict.keys()

In [ ]:
model.representation.interaction[0]

In [ ]:
script(model.representation.interactions[0].conv.tp)

In [ ]:
task = task.load_from_checkpoint(model_path, model=model, strict=False)

In [ ]:
task

In [ ]:
state_dict = torch.load(model_path)

In [ ]:
state_dict['state_dict']['model.output_modules.2.atomic_energies'] = torch.zeros((119,), dtype=torch.float, device='cuda:0')

In [ ]:
torch.save(state_dict, model_path)

In [ ]:
model.cutoff

In [ ]:
cfg

In [ ]:
datamodule = instantiate(cfg.data)

In [ ]:
datamodule.setup()

In [ ]:
optim = instantiate(cfg.task.optimizer)(model.parameters())

In [ ]:
sample = next(iter(datamodule.train_dataset))

In [ ]:
sample["_node_feat"].shape

In [ ]:
sample["_node_attr"].shape

In [ ]:
model.representation.interactions[1].skip_tp

In [ ]:
model(sample)

In [ ]:
model = instantiate(cfg.model)

In [ ]:
optimizer = torch.optim.Adam(
    net.parameters(), 
    lr=args.initial_lr,
)
criterion = torch.nn.MSELoss()

In [ ]:
task = instantiate(cfg.task, model=model)

In [ ]:
trainer = instantiate(cfg.trainer)

In [ ]:
trainer.fit(task, datamodule=datamodule)

In [ ]:
file_path = "models/Au_100/96_node_3_layer.pth"
num_interactions = int(file_path.split('/')[-1].split('_')[2])
num_features = int(file_path.split('/')[-1].split('_')[0])

cfg.model.representation.num_features = num_features
cfg.model.representation.num_interactions = num_interactions
model = instantiate(cfg.model)

# load old model
old_model = PainnModel(
    num_interactions=num_interactions,
    node_size=num_features,
    cutoff=5.0,
)
state_dict = torch.load(file_path) 
old_model.load_state_dict(state_dict["model"])

# load new model
cfg.model.representation.num_features = num_features
cfg.model.representation.num_interactions = num_interactions
model = instantiate(cfg.model)

representation = OrderedDict()
del_keys = ['normalization', 'atomwise_normalization', 'normalize_stddev', 'normalize_mean']
for k, v in state_dict["model"].items():
    if k not in del_keys:
        representation[k] = v

rescale = OrderedDict([
    ('atomwise_normalization', state_dict["model"]['atomwise_normalization']),
    ('scale', state_dict["model"]['normalize_stddev'].unsqueeze(-1)),
    ('shift', state_dict["model"]['normalize_mean'].unsqueeze(-1)),
])

model.representation.load_state_dict(representation)
model.output_modules[1].load_state_dict(rescale, strict=False)
compiled_model = script(model)
metadata = {"cutoff": str(compiled_model.representation.cutoff).encode("ascii")}
compiled_model.save(f"models/compiled_model/{num_features}_node_{num_interactions}_layer.pt", _extra_files=metadata)

In [ ]:
class EnsembleModel(nn.Module):
    """
    Ensemble model of PaiNN. This is used to get uncertainty informations.
    """
    def __init__(self, models: List[NeuralNetworkPotential]) -> None:
        super().__init__()
        self.models = nn.ModuleList([model for model in models])

    def forward(self, data: properties.Type) -> properties.Type:
        energy = []
        forces = []
        for model in self.models:
            out = model(data)
            energy.append(out[properties.energy].detach())
            forces.append(out[properties.forces].detach())
        
        energy = torch.stack(energy)
        forces = torch.stack(forces)
        f_scatter = torch.zeros(data[properties.n_atoms].shape[0], device=out[properties.energy].device)
        result_dict ={
            'energy': torch.mean(energy, dim=0),
            'forces': torch.mean(forces, dim=0),
            'e_var': torch.var(energy, dim=0),
            'e_sd': torch.std(energy, dim=0),
            'f_var': f_scatter.index_add(0, data[properties.image_idx], torch.var(forces, dim=0).mean(dim=1)) / data[properties.n_atoms],
        }
        
        result_dict['f_sd'] = result_dict['f_var'].sqrt()
        if 'energy' in data.keys():
            e_diff = result_dict['energy'] - data['energy']
            f_diff = result_dict['forces'] - data['forces']
            result_dict['e_ae'] = torch.abs(e_diff)
            result_dict['e_se'] = torch.square(e_diff)
            result_dict['f_ae'] = f_scatter.index_add(0, data[properties.image_idx], torch.abs(f_diff).mean(dim=1)) / data[properties.n_atoms]
            result_dict['f_se'] = f_scatter.index_add(0, data[properties.image_idx], torch.square(f_diff).mean(dim=1)) / data[properties.n_atoms]

        return result_dict 

In [ ]:
models = []
for path in Path('models').rglob('compiled_model/*.pt'):
    models.append(torch.jit.load(path))

In [ ]:
ensemble = EnsembleModel(models)

In [ ]:
datamodule.setup()
batch = next(iter(datamodule.train_dataset))
new_batch = copy.deepcopy(batch)

batch["num_atoms"] = batch.pop("_n_atoms")
batch["elems"] = batch.pop("_atomic_number")
batch["coord"] = batch.pop("_positions")
batch["pairs"] = batch.pop("_edge_idx")
batch["n_diff"] = batch.pop("_edge_diff")
batch["cell"] = batch.pop("_cell")
batch["num_pairs"] = batch.pop("_n_pairs")

# old_results = old_model(batch)
# new_results = model(new_batch)
# torch.allclose(old_results["forces"], new_results["forces"], atol=1e-4)

In [ ]:
nequip_model = torch.jit.load("/home/xinyang/deployed_model.pth")

In [ ]:
with open("/home/xinyang/full.yaml") as f:
    config = yaml.safe_load(f)

In [ ]:
aspirin = np.load('./benchmark_data/small_aspirin_dataset.npz')
np.savez(
    "small_aspirin_dataset.npz", 
    E=aspirin["E"][:1000],
    name=aspirin["name"],
    F=aspirin["F"][:1000],
    theory=aspirin["theory"],
    R=aspirin["R"][:1000],
    z=aspirin["z"],
    type=aspirin["type"],
    md5=aspirin["md5"],
)

In [ ]:
model.representation.embeddings.radial_basis.load_state_dict(nequip_model.model.func.radial_basis.state_dict())
model.representation.embeddings.chemical_embedding.load_state_dict(nequip_model.model.func.chemical_embedding.state_dict())

model.representation.interactions[0].load_state_dict(nequip_model.model.func.layer0_convnet.state_dict(), strict=False)
model.representation.interactions[1].load_state_dict(nequip_model.model.func.layer1_convnet.state_dict(), strict=False)
model.representation.interactions[2].load_state_dict(nequip_model.model.func.layer2_convnet.state_dict(), strict=False)
model.representation.interactions[3].load_state_dict(nequip_model.model.func.layer3_convnet.state_dict(), strict=False)
model.representation.interactions[4].load_state_dict(nequip_model.model.func.layer4_convnet.state_dict(), strict=False)


model.representation.readout_mlp[0].load_state_dict(nequip_model.model.func.conv_to_output_hidden.linear.state_dict())
model.representation.readout_mlp[1].load_state_dict(nequip_model.model.func.output_hidden_to_scalar.linear.state_dict())

model.representation.interactions[0].conv.avg_num_neighbors = torch.tensor([nequip_model.model.func.layer0_convnet.conv.avg_num_neighbors])
model.representation.interactions[1].conv.avg_num_neighbors = torch.tensor([nequip_model.model.func.layer1_convnet.conv.avg_num_neighbors])
model.representation.interactions[2].conv.avg_num_neighbors = torch.tensor([nequip_model.model.func.layer2_convnet.conv.avg_num_neighbors])
model.representation.interactions[3].conv.avg_num_neighbors = torch.tensor([nequip_model.model.func.layer3_convnet.conv.avg_num_neighbors])
model.representation.interactions[4].conv.avg_num_neighbors = torch.tensor([nequip_model.model.func.layer4_convnet.conv.avg_num_neighbors])

model.output_modules[-1].scale = nequip_model.scale_by.unsqueeze(-1)
model.output_modules[-1].shift = nequip_model.shift_by.unsqueeze(-1)
model.output_modules[-1].has_scale = True
model.output_modules[-1].has_shift = True
model.output_modules[-1].atomwise_normalization = torch.tensor(False)

In [ ]:
dataset = dataset_from_config(config)
datamodule.setup()

In [ ]:
sample = dataset[0]
sample = AtomicData.to_AtomicDataDict(sample)
sample_2 = next(iter(datamodule.dataset))

In [ ]:
model(sample_2)

In [ ]:
results = nequip_model(sample)

In [ ]:
results['total_energy']

In [ ]:
modules = []
modules.append(model.input_modules[0])
for k, v in model.representation.embeddings.items():
    modules.append(v)
modules.append(model.representation.interactions[0])
modules.append(model.representation.interactions[1])
modules.append(model.representation.interactions[2])
modules.append(model.representation.interactions[3])
modules.append(model.representation.interactions[4])

In [ ]:
for m in modules:
    sample_2 = m(sample_2)

sample_2[properties.atomic_energy] = model.representation.readout_mlp(sample_2[properties.node_feat]).squeeze()
sample_2 = model.output_modules[0](sample_2)
sample_2 = model.output_modules[1](sample_2)
# sample_2 = model.output_modules[2](sample_2)

In [ ]:
sample_2["energy"]

In [ ]:
embeddings = [
    nequip_model.model.func.one_hot,
    nequip_model.model.func.spharm_edges,
    nequip_model.model.func.radial_basis,
    nequip_model.model.func.chemical_embedding,
    nequip_model.model.func.layer0_convnet,
    nequip_model.model.func.layer1_convnet,
    nequip_model.model.func.layer2_convnet,
    nequip_model.model.func.layer3_convnet,
    nequip_model.model.func.layer4_convnet,
    nequip_model.model.func.conv_to_output_hidden,
    nequip_model.model.func.output_hidden_to_scalar,
    # nequip_model.model.func.per_species_rescale,
    nequip_model.model.func.total_energy_sum,
]

In [ ]:
for m in embeddings:
    sample = m(sample)

In [ ]:
sample['total_energy']

In [ ]:
for m in nequip_model.named_modules():
    print(m[0])

In [ ]:
update_metrics

In [ ]:
results = nequip_model(sample)

In [ ]:
sample_2['energy']

In [ ]:
sample["total_energy"]

In [ ]:
nequip_model.model.func

In [ ]:
sample_2["energy"]

In [ ]:
sample = nequip_model.model(sample)

In [ ]:
results["total_energy"]

In [ ]:
sample_2["energy"]

In [ ]:
sample["total_energy"] * nequip_model.scale_by + nequip_model.shift_by

In [ ]:
sample_2[properties.node_feat]

In [ ]:
sample[AtomicDataDict.NODE_FEATURES_KEY]

In [ ]:
from e3nn.nn import FullyConnectedNet
from curator.layer import ShiftedSoftPlus

In [ ]:
fc = FullyConnectedNet(
    [8, 64, 64, 64],
    {
        "ssp": ShiftedSoftPlus,
        "silu": torch.nn.functional.silu,
    }["silu"],
)

In [ ]:
fc.load_state_dict(nequip_model.model.func.layer0_convnet.conv.fc.state_dict())

In [ ]:
fc(sample_2[properties.edge_dist_embedding])

In [ ]:
nequip_model.model.func.layer0_convnet.conv.fc(sample_2[properties.edge_dist_embedding])

In [ ]:
weight = model.representation.interactions[0].conv.fc(sample_2[properties.edge_dist_embedding])

x = sample_2[properties.node_feat]
edge_idx = sample_2[properties.edge_idx]  # i, j index

if model.representation.interactions[0].conv.sc is not None:
    sc = model.representation.interactions[0].conv.sc(x, sample_2[properties.node_attr])

x = model.representation.interactions[0].conv.linear_1(x)
edge_features_2 = model.representation.interactions[0].conv.tp(
    x[edge_idx[:, 1]], sample_2[properties.edge_diff_embedding], weight
)
x = scatter_add(edge_features_2, edge_idx[:, 0], dim_size=len(x), dim=0)

# Necessary to get TorchScript to be able to type infer when its not None
# avg_num_neigh: Optional[float] = self.avg_num_neighbors
# if avg_num_neigh is not None:
x = x.div(model.representation.interactions[0].conv.avg_num_neighbors**0.5)

x = model.representation.interactions[0].conv.linear_2(x)

if model.representation.interactions[0].conv.sc is not None:
    x = x + sc

sample_2[properties.node_feat] = x

In [ ]:
weight = nequip_model.model.func.layer0_convnet.conv.fc(sample[AtomicDataDict.EDGE_EMBEDDING_KEY])

x = sample[AtomicDataDict.NODE_FEATURES_KEY]
edge_src = sample[AtomicDataDict.EDGE_INDEX_KEY][1]
edge_dst = sample[AtomicDataDict.EDGE_INDEX_KEY][0]

if nequip_model.model.func.layer0_convnet.conv.sc is not None:
    sc = nequip_model.model.func.layer0_convnet.conv.sc(x, sample[AtomicDataDict.NODE_ATTRS_KEY])

x = nequip_model.model.func.layer0_convnet.conv.linear_1(x)
edge_features = nequip_model.model.func.layer0_convnet.conv.tp(
    x[edge_src], sample[AtomicDataDict.EDGE_ATTRS_KEY], weight
)
x = scatter(edge_features, edge_dst, dim=0, dim_size=len(x))

# Necessary to get TorchScript to be able to type infer when its not None
avg_num_neigh: Optional[float] = nequip_model.model.func.layer0_convnet.conv.avg_num_neighbors
if avg_num_neigh is not None:
    x = x.div(avg_num_neigh**0.5)

x = nequip_model.model.func.layer0_convnet.conv.linear_2(x)

if nequip_model.model.func.layer0_convnet.conv.sc is not None:
    x = x + sc

sample[AtomicDataDict.NODE_FEATURES_KEY] = x

In [12]:
import torch
from torch import nn
import torch.nn.functional as F
from torch_ema import ExponentialMovingAverage
from typing import List, Optional, Dict, Type, Any
from curator.data import properties
from pytorch_lightning.utilities.types import STEP_OUTPUT
import warnings
import pytorch_lightning as pl
import torchmetrics
from torchmetrics import Metric
import copy
from pytorch_lightning import LightningDataModule
from omegaconf import DictConfig
import logging
from collections import OrderedDict, defaultdict

In [13]:
class ModelOutput(nn.Module):
    """ Base class for model outputs."""
    def __init__(
        self,
        name: str,
        loss_fn: Optional[nn.Module] = None,
        loss_weight: float = 1.0,
        metrics: Optional[Dict[str, Metric]] = None,
        target_property: Optional[str]=None,
    ) -> None:
        """ Base class for model outputs. 

        Args:
            name (str): Name of the output
            loss_fn (Optional[nn.Module], optional): Loss function. Defaults to None.
            loss_weight (float, optional): Loss weight. Defaults to 1.0.
            metrics (Optional[Dict[str, Metric]], optional): Metrics. Defaults to None.
            target_property (Optional[str], optional): Target property. Defaults to None.
        """
        super().__init__()
        self.name = name
        self.target_property = target_property or name
        self.loss_fn = loss_fn
        self.loss_weight = loss_weight
        self.train_metrics = nn.ModuleDict(metrics)
        self.val_metrics = nn.ModuleDict({k: copy.copy(v) for k, v in metrics.items()})
        self.test_metrics = nn.ModuleDict({k: copy.copy(v) for k, v in metrics.items()})
        
        # here we found a serious bug that deepcopy is not working in hydra instantiate!!!
        self.metrics = {
            "train": self.train_metrics,
            "val": self.val_metrics,
            "test": self.test_metrics,
        }
        
        # accumulated loss
        self.loss = defaultdict(lambda: 0.0)
        self.num_obs = defaultdict(lambda: 0)
    
    def calculate_loss(self, pred: Dict, target: Dict, return_num_obs=True, subset: str='train') -> torch.Tensor:
        if self.loss_weight == 0 or self.loss_fn is None:
            return 0.0

        loss = self.loss_weight * self.loss_fn(
            pred[self.name], target[self.target_property]
        )
        num_obs = target[self.target_property].view(-1).shape[0]

        self.loss[subset] += loss.detach()
        self.num_obs[subset] += num_obs
        
        if return_num_obs:
            return loss, num_obs
        
        return loss

    def calculate_metrics(self, pred: Dict, target: Dict, subset: str) -> None:
        batch_val = OrderedDict()
        for k in self.metrics[subset]:
            batch_val[f"{subset}_{self.name}_{k}"] = self.metrics[subset][k](pred[self.name], target[self.target_property])
        return batch_val
    
    def reset_loss(self, subset: Optional[str]=None) -> None:
        if subset is None:
            for k in self.loss:
                self.loss[k] = 0.0
                self.num_obs[k] = 0
        else:
            self.loss[subset] = 0.0
            self.num_obs[subset] = 0
    
    def reset_metrics(self, subset: Optional[str]=None) -> None:
        if subset is None:
            for k1 in self.metrics:
                for k2 in self.metrics[k1]:
                    self.metrics[k1][k2].reset()
        else:
            for k in self.metrics[subset]:
                self.metrics[subset][k].reset()